Build a regression model.

In [17]:
import pandas as pd
import statsmodels.api as sm
df_cleaned = pd.read_csv('combine_df.csv')
df_cleaned.head()


,station_id,name,latitude,longitude,free_bikes,empty_slots,POI_name,rating,review_count,POI_lat,POI_lon,category,address,POI_name_foursquare,POI_lat_foursquare,POI_lon_foursquare,category_foursquare,address_foursquare
0,019fbdf044ddeded277660f9d9c5fb23,E Bethune St & Brush St,42.374231,-83.069759,1,10,Oak & Reel,4.5,288,42.372278,-83.066819,Italian,"2921 E Grand Blvd, Detroit, MI 48202",Tuff Taco,42.34424,-83.157737,Restaurant,"10005 W Warren Ave, Dearborn, MI 48126"
1,019fbdf044ddeded277660f9d9c5fb23,E Bethune St & Brush St,42.374231,-83.069759,1,10,Oak & Reel,4.5,288,42.372278,-83.066819,Italian,"2921 E Grand Blvd, Detroit, MI 48202",Tuff Taco,42.34424,-83.157737,Restaurant,"10005 W Warren Ave, Dearborn, MI 48126"
2,019fbdf044ddeded277660f9d9c5fb23,E Bethune St & Brush St,42.374231,-83.069759,1,10,Oak & Reel,4.5,288,42.372278,-83.066819,Italian,"2921 E Grand Blvd, Detroit, MI 48202",Tuff Taco,42.34424,-83.157737,Restaurant,"10005 W Warren Ave, Dearborn, MI 48126"
3,019fbdf044ddeded277660f9d9c5fb23,E Bethune St & Brush St,42.374231,-83.069759,1,10,Oak & Reel,4.5,288,42.372278,-83.066819,Italian,"2921 E Grand Blvd, Detroit, MI 48202",Tuff Taco,42.34424,-83.157737,Restaurant,"10005 W Warren Ave, Dearborn, MI 48126"
4,019fbdf044ddeded277660f9d9c5fb23,E Bethune St & Brush St,42.374231,-83.069759,1,10,Oak & Reel,4.5,288,42.372278,-83.066819,Italian,"2921 E Grand Blvd, Detroit, MI 48202",Tuff Taco,42.34424,-83.157737,Restaurant,"10005 W Warren Ave, Dearborn, MI 48126"


In [19]:
#POI's near each station
poi_counts = df_cleaned.groupby('station_id')['POI_name_foursquare'].count().reset_index(name='POIs_nearby')

#yelp reviews per station
yelp_reviews = df_cleaned.groupby('station_id')['review_count'].sum().reset_index(name='yelp_review_count')

#avg Yelp rating per station
yelp_rating = df_cleaned.groupby('station_id')['rating'].mean().reset_index(name='avg_yelp_rating')

#combine all features
df_features = poi_counts.merge(yelp_reviews, on='station_id').merge(yelp_rating, on='station_id')


In [21]:
#average number of free bikes per station
free_bikes = df_cleaned.groupby('station_id')['free_bikes'].mean().reset_index()

#merge features with target
df_model = df_features.merge(free_bikes, on='station_id')
df_model.head()


,station_id,POIs_nearby,yelp_review_count,avg_yelp_rating,free_bikes
0,019fbdf044ddeded277660f9d9c5fb23,400,47660,4.155,1.0
1,021d0e9c8fcd21ec32bd3136945ef332,400,175560,4.220,5.0
2,0ad79e03add4ffef597d9a7b51d374be,400,22500,3.475,5.0
3,0d077ef68550428f754b9685b5750b27,400,161620,4.260,5.0
4,0d7d5ce52fd3c9eea183caed919e0dd1,400,79780,4.220,4.0


In [23]:
#define features and target
X = df_model[['POIs_nearby', 'yelp_review_count', 'avg_yelp_rating']]
y = df_model['free_bikes']

#constant (for intercept)
X = sm.add_constant(X)

#build + fit the model
model = sm.OLS(y, X).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.6692
Date:                Thu, 10 Jul 2025   Prob (F-statistic):              0.574
Time:                        17:48:14   Log-Likelihood:                -180.14
No. Observations:                  79   AIC:                             368.3
Df Residuals:                      75   BIC:                             377.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 7.5292      3.14

Provide model output and an interpretation of the results. 

The regression model attempts to predict the number of free bikes using POIs nearby, total number of Yelp reviews, and average Yelp rating (avg_yelp_rating). 

R-squared =0.026
- This means the  model only explains about 2.6% of the variability in free_bikes
- This is very low. This suggests that current features are not strongly correlated with bike availability prediction.

p-values for all predictors > 0.05
- This means none of the predictors are statistically significant in the model (cannot confidently say that they impact free_bikes)

Breaking down the coefficients
- POIs_nearby= -0.0029	(shows a slight negative relationship; indicates more POIs may slightly reduce free bikes)
- yelp_review_count= 0.0000035	(shows a weak positive trend; more reviews suggest more bikes) 
- avg_yelp_rating= -0.7437 (shows higher-rated areas have fewer free bikes)

# Stretch

How can you turn the regression model into a classification model?